In [204]:
import pandas as pd
import numpy as np

In [205]:
# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

In [206]:
# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'country', 'rgdpna', 'rkna', 'emp','avh', 'labsh','rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

In [207]:
# Calculate additional columns
data['L'] = data['emp'] * data['avh']  # L
data['ln_y'] = np.log(data['rgdpna'] / data['L']) # Y/L
data['ln_A'] = np.log(data['rtfpna']) # A
data['ln_k'] = np.log(data['rkna'] / data['L'])  # K/L
data['alpha'] = 1 - data['labsh']  # alpha

In [208]:
# Order by year
data = data.sort_values('year')

In [209]:
# Group by isocode
grouped_data = data.groupby('country')

In [210]:
# Remove missing values
data = data.dropna()

In [211]:
# Calculate growth rates and Solow residual
data['g_rate'] = (grouped_data['ln_y'].diff() * 100)  # Growth rate of GDP 
data['tfp_g'] = (grouped_data['ln_A'].diff() * 100)  # TFP Growth
data['cap_d'] = data['alpha'] * (grouped_data['ln_k'].diff() * 100) # Capital Deepening
data['tfp_s'] = data['tfp_g'] / data['g_rate']  #TFP Share

In [212]:
# Calculate summary statistics
summary = data.groupby('country').agg({'g_rate': 'mean','tfp_g': 'mean','cap_d': 'mean','tfp_s': 'mean','alpha': 'mean'})

In [213]:
# Calculate additional summary statistics
summary['Growth Rate'] = summary['g_rate']
summary['TFP Growth'] = summary['tfp_g']
summary['Capital Deepening'] = summary['cap_d']
summary['TFP Share'] = summary['tfp_s']
summary['Capital Share'] = summary['alpha']

In [214]:
# Print output
print(summary.iloc[:, -5:])

                Growth Rate  TFP Growth  Capital Deepening  TFP Share  \
country                                                                 
Canada             1.117384    0.151360           0.713034   0.504413   
France             1.087678    0.093816           0.666056   0.315030   
Germany            1.073874    0.358568           0.584524   0.367890   
Italy              0.285631   -0.772207           0.689669   0.508958   
Japan              1.221666    0.142913           0.848135  -1.074495   
United Kingdom     1.149943    0.230807           0.628362   0.730544   
United States      1.658237    0.675462           0.825947   0.574376   

                Capital Share  
country                        
Canada               0.345020  
France               0.381140  
Germany              0.369455  
Italy                0.484425  
Japan                0.429427  
United Kingdom       0.414704  
United States        0.393235  


In [215]:
# Equal to 1?
summary['TFP Growth + Capital Deepening'] = summary['TFP Growth'] + summary['Capital Deepening']
summary['TFP Share + Capital Share'] = summary['TFP Share'] + summary['Capital Share']

In [216]:
print(summary.iloc[:, -2:])

                TFP Growth + Capital Deepening  TFP Share + Capital Share
country                                                                  
Canada                                0.864394                   0.849433
France                                0.759872                   0.696171
Germany                               0.943092                   0.737346
Italy                                -0.082537                   0.993383
Japan                                 0.991048                  -0.645069
United Kingdom                        0.859169                   1.145248
United States                         1.501409                   0.967611
